In [1]:
using LinearAlgebra

In [2]:
function conjgrad1(A::Matrix{Float64}, b::Vector{Float64}, x::Vector{Float64})

    r = A * x - b
    p = -r
        
    k = 1
    while norm(r) > 10e-6
        # @show k, norm(r)
        α = -(r' * p) / (p' * A * p)
        x = x + α * p
        r = A * x - b
        β = (r' * A * p) / (p' * A * p)
        p = -r + β * p
        k += 1
    end
    
    return x, k

end

conjgrad1 (generic function with 1 method)

In [3]:
function conjgrad2(A::Matrix{Float64}, b::Vector{Float64}, x::Vector{Float64})
    
    r = A * x - b
    p = -r
        
    k = 1
    while norm(r) > 10e-6
        # @show k, norm(r)
        α = (r' * r) / (p' * A * p)
        x = x + α * p
        
        r_old = deepcopy(r)
        r = r_old + α * A * p
        
        β = (r' * r) / (r_old' * r_old)
        p = -r + β * p
        k += 1
    end
    
    return x, k

end

conjgrad2 (generic function with 1 method)

In [4]:
A(n::Int64) = [1 / (i + j - 1) for i=1:n, j=1:n]  # Hilbert matrix
b(n::Int64) = ones(n)  # Target
x(n::Int64) = zeros(n)  # Initial guess

x (generic function with 1 method)

In [5]:
using BenchmarkTools

for n in [5, 8, 12, 20]
    @show n
    _, k₁ = @btime conjgrad1(A($n), b($n), x($n))
    _, k₂ = @btime conjgrad2(A($n), b($n), x($n))
    @show k₁, k₂
end

n = 5
  2.111 μs (66 allocations: 6.34 KiB)
  1.967 μs (72 allocations: 7.75 KiB)
(k₁, k₂) = (7, 7)
n = 8
  10.625 μs (286 allocations: 36.19 KiB)
  6.667 μs (215 allocations: 28.80 KiB)
(k₁, k₂) = (29, 20)
n = 12
  14.791 μs (316 allocations: 50.44 KiB)
  8.819 μs (237 allocations: 38.42 KiB)
(k₁, k₂) = (32, 22)
n = 20
  52.250 μs (726 allocations: 161.84 KiB)
  20.167 μs (380 allocations: 82.44 KiB)
(k₁, k₂) = (73, 35)
